In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import time

### Loading Data and Processing

In [2]:
#############Loading raw data#############
df = pd.read_csv("data/flow_s_alltime_Dropna.csv")
df.shape[0]

993923

In [3]:
#############Loading Label#############
dflabel_blstm = pd.read_csv("Results/ex_label&model-blstm.csv")
dflabel_blstm_cnn = pd.read_csv("Results/ex_label&model-blstm-cnn.csv")
dflabel_cnn = pd.read_csv("Results/ex_label&model-cnn.csv")
dflabel_cnn_blstm = pd.read_csv("Results/ex_label&model-cnn-blstm.csv")

dflabel_tfxgb = pd.read_csv("Results/ex_label_tf_xgb.csv")
dflabel_tfrf = pd.read_csv("Results/ex_label_tf_rf.csv")
dflabel_tflsvm = pd.read_csv("Results/ex_label_tf_lsvm.csv")

dflabel_wvxgb = pd.read_csv("Results/ex_label_wv_xgb.csv")
dflabel_wvrf = pd.read_csv("Results/ex_label_wv_rf.csv")
dflabel_wvlsvm = pd.read_csv("Results/ex_label_wv_lsvm.csv")

dflabel_blstm.shape[0]

993923

In [4]:
#############Concat#############
dfconcat_blstm = pd.concat([df, dflabel_blstm], axis=1)
dfconcat_blstm_cnn = pd.concat([df, dflabel_blstm_cnn], axis=1)
dfconcat_cnn = pd.concat([df, dflabel_cnn], axis=1)
dfconcat_cnn_blstm = pd.concat([df, dflabel_cnn_blstm], axis=1)

dfconcat_tfxgb = pd.concat([df, dflabel_tfxgb['label1']], axis=1)
dfconcat_tfrf = pd.concat([df, dflabel_tfrf['label1']], axis=1)
dfconcat_tflsvm = pd.concat([df, dflabel_tflsvm['label1']], axis=1)

dfconcat_tfxgb.rename(columns={'label1':'label'},inplace=True) 
dfconcat_tfrf.rename(columns={'label1':'label'},inplace=True) 
dfconcat_tflsvm.rename(columns={'label1':'label'},inplace=True) 

dfconcat_wvxgb = pd.concat([df, dflabel_wvxgb['label1']], axis=1)
dfconcat_wvrf = pd.concat([df, dflabel_wvrf['label1']], axis=1)
dfconcat_wvlsvm = pd.concat([df, dflabel_wvlsvm['label1']], axis=1)

dfconcat_wvxgb.rename(columns={'label1':'label'},inplace=True) 
dfconcat_wvrf.rename(columns={'label1':'label'},inplace=True) 
dfconcat_wvlsvm.rename(columns={'label1':'label'},inplace=True) 

In [5]:
#############Importance! sort_values and reset_index#############
dfconcat_blstm = dfconcat_blstm.sort_values(["label"],ascending=True)
dfconcat_blstm_cnn = dfconcat_blstm_cnn.sort_values(["label"],ascending=True)
dfconcat_cnn = dfconcat_cnn.sort_values(["label"],ascending=True)
dfconcat_cnn_blstm = dfconcat_cnn_blstm.sort_values(["label"],ascending=True)
dfconcat_tfxgb = dfconcat_tfxgb.sort_values(["label"],ascending=True)###
dfconcat_tfrf = dfconcat_tfrf.sort_values(["label"],ascending=True)###
dfconcat_tflsvm = dfconcat_tflsvm.sort_values(["label"],ascending=True)###
dfconcat_wvxgb = dfconcat_wvxgb.sort_values(["label"],ascending=True)###
dfconcat_wvrf = dfconcat_wvrf.sort_values(["label"],ascending=True)###
dfconcat_wvlsvm = dfconcat_wvlsvm.sort_values(["label"],ascending=True)###

dfconcat_blstm.reset_index(drop=True, inplace=True)
dfconcat_blstm_cnn.reset_index(drop=True, inplace=True)
dfconcat_cnn.reset_index(drop=True, inplace=True)
dfconcat_cnn_blstm.reset_index(drop=True, inplace=True)
dfconcat_tfxgb.reset_index(drop=True, inplace=True)
dfconcat_tfrf.reset_index(drop=True, inplace=True)
dfconcat_tflsvm.reset_index(drop=True, inplace=True)
dfconcat_wvxgb.reset_index(drop=True, inplace=True)
dfconcat_wvrf.reset_index(drop=True, inplace=True)
dfconcat_wvlsvm.reset_index(drop=True, inplace=True)


In [6]:
dfconcat_wvlsvm.head()
dfconcat_wvrf.head()
dfconcat_wvxgb.head()
dfconcat_tflsvm.head()
dfconcat_tfrf.head()
dfconcat_tfxgb.head()
dfconcat_blstm_cnn.head()

,time,source_ip,destination_ip,num,url,label
0,2019-06-13 10,10.49.212.162,10.49.223.194,1,/index.php?fulltext=%E6%90%9C%E7%B4%A2&profile...,0.0
1,2019-06-13 10,10.49.212.162,10.49.223.194,1,/load.php?*&debug=false&lang=zh-cn&modules=med...,0.0
2,2019-06-06 10,10.56.68.148,10.49.253.175,1,http://www.proxifier.com/distr/ProxifierSetup.exe,0.0
3,2019-06-13 10,10.49.212.162,10.49.223.194,1,/index.php?action=historysubmit&diff=............,0.0
4,2019-06-05 06,10.49.212.7,10.49.216.62,1,/mas/schedule.jsp?type=group&SGPID=1'+UNION+AL...,0.0


### Defining Parameters and Functions

In [7]:
#############Defining Parameters and Functions#############
dfabcorrect=['10.49.223.194','10.49.231.206','10.49.212.126','123.58.173.111', '10.49.212.162','10.49.21.15',
             '10.49.253.233','10.49.253.118','10.49.253.147','10.49.253.204','10.49.253.210','10.49.253.225',
             '10.49.253.252','10.49.253.35','10.49.253.86','10.49.253.87','10.56.144.126','10.49.253.175',
             '10.56.34.63','10.49.253.157','10.56.148.238','10.56.4.86','10.56.68.148','10.56.115.19',
             '10.24.139.131','10.49.126.199']

################Change here to test the results of data leakage incidents identification by web front-end anomaly detection only
# dfabcorrect=['10.49.253.233','10.49.253.35','10.49.223.194','10.49.212.162','10.49.21.15',
#             '10.49.141.206','123.58.173.111']


dfabcorrect_set = set(dfabcorrect)

#######Normalization Function
min_max_scaler_01 = preprocessing.MinMaxScaler(feature_range=(0,1), copy=True)

#######Pruning Function
def Pruning(modelpred_,DelNum):
    if DelNum == 0:
        return modelpred_
    
    modelpred__For_Pruning = modelpred_[modelpred_['label']<0.5][['time','source_ip','destination_ip','num']]
    
    ab_finalsort_half = modelpred__For_Pruning
    ################Important  Converts to strings to match records
    ab_finalsort_half['source_ip'] = ab_finalsort_half['source_ip'].apply(str)
    ab_finalsort_half['destination_ip'] = ab_finalsort_half['destination_ip'].apply(str)
    ab_finalsort_half = ab_finalsort_half[ab_finalsort_half['destination_ip'].str.startswith('10.')]
    
    column_uniques_sip = ab_finalsort_half['source_ip'].unique()
    groups_sip = ab_finalsort_half.groupby('source_ip')

    dfkeep = pd.DataFrame(columns=('sip','tihNum','dipNum','numSum'))

    for column_sip in column_uniques_sip:
        dfexpi_sip = groups_sip.get_group(column_sip)
        dfexpi_sip.reset_index(drop=True, inplace=True)
        dfkeep = dfkeep.append({'sip':column_sip,'tihNum':dfexpi_sip['time'].nunique(),'dipNum':dfexpi_sip['destination_ip'].nunique(),'numSum':dfexpi_sip['num'].sum()}, ignore_index=True)

    mmfeature=[x for x in dfkeep.columns if x not in ['sip']]
    for col in dfkeep[mmfeature]:
        dfkeep[[col]]=min_max_scaler_01.fit_transform(dfkeep[[col]])
    
    ###Remove sips that do not meet multi-tih and multi-dip
    dfkeep = dfkeep.drop(index = dfkeep[dfkeep['dipNum'] == 0].index.tolist())
    dfkeep = dfkeep.drop(index = dfkeep[dfkeep['tihNum'] == 0].index.tolist())
    
    dfkeep['nor_score'] = dfkeep['tihNum'] + dfkeep['dipNum'] + dfkeep['numSum']
    dfkeep.sort_values(by="nor_score" , inplace=True, ascending=False)
    dfkeep.reset_index(drop=True, inplace=True)

    dfkeep["nor_score"] = pd.to_numeric(dfkeep["nor_score"],errors='coerce')

    dfdrop_ip = dfkeep[0:DelNum]['sip']
    dfdrop_iplist = dfdrop_ip.tolist()
    
    for ip in dfdrop_iplist:
        modelpred_ = modelpred_.drop(index = modelpred_[modelpred_['source_ip'] == ip].index.tolist())###Delete Scanner-related records
    modelpred_.reset_index(drop=True, inplace=True)
    finalkeep = modelpred_ 
    return finalkeep

#######Metric Function
def Metric(IPnum,abIP,TP,FN):
    FP=abIP-TP
    TN=IPnum-abIP-FN
    R= TP/(TP+FN)
    TPR = TP/(TP+FN)
    FPR = FP/(TN+FP)
    P= TP/(TP+FP)
    return round(R, 4),round(TPR, 4),round(FPR, 4),round(P, 4)

### Batch Experimental Testing

In [8]:
ModelList = [dfconcat_blstm,dfconcat_blstm_cnn,dfconcat_cnn,dfconcat_cnn_blstm,dfconcat_tfxgb,dfconcat_tfrf,dfconcat_tflsvm,dfconcat_wvxgb,dfconcat_wvrf,dfconcat_wvlsvm]
# ModelList = [dfconcat_blstm_cnn]

In [11]:
#######Define logdf to store logs for visualization
Logdf = pd.DataFrame(columns=('ModelName','DelNum','ROC_AUC','PRC_AUC','TimeCost'))

IPnumlist = pd.concat([dfconcat_blstm["source_ip"], dfconcat_blstm["destination_ip"]], axis=0)
IPnumlist=IPnumlist.drop_duplicates()
IPnumlist.reset_index(drop=True, inplace=True)
IPnum= IPnumlist.shape[0] ##IP Num should be the same

ModelName = 0
for modelpred in ModelList:
    ModelName += 1###Model name ID, from 1 to 10, represents the above models in order

    for i in range(0,7):######Test for 0-6 ###### Of course you can increase the number of scanners to be excluded, but the results will not be better
        time_start = time.time() 

        #######Define Lists to keep TPR FPR R P
        TPRlist = []
        FPRlist = []
        Plist = []
        Rlist = []
        #######Pruning or not by changing i
        finalkeep = Pruning(modelpred,i)

        num = 1 ########Draw ROC and PRC curves according to the selection of num, That is, the number of considered abnormal records
        #When num is 0, i.e. all records are considered normal  
        #At this point R TPR FPR are 0, and P is not defined  
        #So we define num to start from 1
        while num < finalkeep.shape[0]+50:###+50 Ensure that all records can be obtained
            ab_finalsort= finalkeep[0:num]
            num += 50
            dfabip=pd.concat([ab_finalsort["source_ip"], ab_finalsort["destination_ip"]], axis=0)
            dfabip=dfabip.drop_duplicates()
            dfabip.reset_index(drop=True, inplace=True)
            if len(dfabip) == 0:
                continue
            dfabip.tolist()
            dfabip_set = set(dfabip)##Predicted set of abnormal IPs
            bingo=(dfabip_set)&(dfabcorrect_set)
            abIPi=len(dfabip_set)####Number of predicted abnormal IPs
            TPi=len(bingo)
            FNi=len(dfabcorrect)-TPi
            R,TPR,FPR,P = Metric(IPnum,abIPi,TPi,FNi)
            TPRlist.append(TPR)
            FPRlist.append(FPR)
            Plist.append(P)
            Rlist.append(R)
        ROC = pd.DataFrame(FPRlist, columns=['FPR'])
        ROC = pd.concat([ROC, pd.DataFrame(TPRlist,columns=['TPR'])],axis=1)
        PR = pd.DataFrame(Rlist, columns=['R'])
        PR = pd.concat([PR, pd.DataFrame(Plist,columns=['P'])],axis=1)

                
        ####Importance drop_duplicates
        ROC=ROC.drop_duplicates()
        PR=PR.drop_duplicates()
        ####Importance sort_values
        ROC.sort_values(by=['FPR','TPR'],inplace=True,ascending=True)
        PR.sort_values(by=['R','P'],inplace=True,ascending=[True,False])#########Same R, P starting from the higher
        ROC.reset_index(drop=True, inplace=True)
        PR.reset_index(drop=True, inplace=True)
        
        #################Add after the Sort
        #####Importance Add the last line For ROC
        ROC.loc[ROC.shape[0]] = [1,ROC.loc[ROC.shape[0]-1,'TPR']]     
        #####Importance Add the first line For PR
        PR.loc[-1] = [0,PR.loc[0,'P']]
        PR.index = PR.index + 1
        PR.sort_index(inplace=True)
        
        #####AUC
        ROC_AUC = np.trapz(list(ROC['TPR']),list(ROC['FPR'])).round(4)
        PRC_AUC = np.trapz(list(PR['P']),list(PR['R'])).round(4)
        
        time_end = time.time() 
        cost = round((time_end - time_start),4)
        
        ######Save Logs
        Path_ROC = "Results/WF_Batch_Pruning/ROC_Model%d_Prun%d.csv"%(ModelName,i)
        ROC.to_csv(Path_ROC,index=0)
        Path_PRC = "Results/WF_Batch_Pruning/PRC_Model%d_Prun%d.csv"%(ModelName,i)
        PR.to_csv(Path_PRC,index=0)
        
#         print(ModelName,i,ROC_AUC,PRC_AUC,cost)
        Logdf = Logdf.append({'ModelName':ModelName,'DelNum':i,'ROC_AUC':ROC_AUC,'PRC_AUC':PRC_AUC,'TimeCost':cost}, ignore_index=True)
#         ModelName i ROC_AUC PRC_AUC cost
########Logdf should have 10*7 records 

1 0 0.9609 0.3421 775.0458
1 1 0.9788 0.5667 301.9403
1 2 0.9872 0.6582 286.9657
1 3 0.988 0.6736 284.9085
1 4 0.949 0.5916 238.9919
1 5 0.9118 0.5788 241.0096
1 6 0.9118 0.5788 243.8687
2 0 0.9662 0.4237 747.7941
2 1 0.9818 0.6032 276.988
2 2 0.9889 0.6844 266.5257
2 3 0.9897 0.6979 269.6081
2 4 0.9509 0.6292 224.1128
2 5 0.9525 0.6587 222.005
2 6 0.915 0.6392 227.6414
3 0 0.9598 0.3716 751.4616
3 1 0.9761 0.5434 282.1911
3 2 0.9847 0.6387 275.9941
3 3 0.9854 0.6511 272.5329
3 4 0.6484 0.4833 271.5308
3 5 0.6094 0.4098 235.3844
3 6 0.5713 0.3839 235.9295
4 0 0.9646 0.4085 724.5987
4 1 0.9793 0.5274 270.7946
4 2 0.9797 0.5294 259.7204
4 3 0.9871 0.6081 254.1095
4 4 0.9481 0.5473 214.1454
4 5 0.949 0.5674 211.2811
4 6 0.9111 0.5325 211.0715
5 0 0.9428 0.3486 714.3534
5 1 0.9582 0.4683 270.2402
5 2 0.96 0.476 259.2321
5 3 0.9691 0.6018 253.736
5 4 0.9299 0.5261 212.3052
5 5 0.9307 0.5364 210.845
5 6 0.8933 0.5085 209.6351
6 0 0.9343 0.3414 728.9899
6 1 0.9497 0.469 280.052
6 2 0.9513 0.4

In [14]:
Logdf

,ModelName,DelNum,ROC_AUC,PRC_AUC,TimeCost
0,1.0,0.0,0.9609,0.3421,775.0458
1,1.0,1.0,0.9788,0.5667,301.9403
2,1.0,2.0,0.9872,0.6582,286.9657
3,1.0,3.0,0.9880,0.6736,284.9085
4,1.0,4.0,0.9490,0.5916,238.9919
5,1.0,5.0,0.9118,0.5788,241.0096
6,1.0,6.0,0.9118,0.5788,243.8687
7,2.0,0.0,0.9662,0.4237,747.7941
8,2.0,1.0,0.9818,0.6032,276.9880
9,2.0,2.0,0.9889,0.6844,266.5257


In [13]:
Logdf.to_csv("Results/WF_Batch_Pruning/All_Logs.csv",index=0)

In [190]:
Logdf[Logdf['ModelName']==1]['ROC_AUC']

0    0.9609
1    0.9788
2    0.9872
3    0.9880
4    0.9490
5    0.9118
6    0.9118
Name: ROC_AUC, dtype: float64

### Extraction for DFADR

In [167]:
modelpred = dfconcat_blstm_cnn
i = 3

IPnumlist = pd.concat([modelpred["source_ip"], modelpred["destination_ip"]], axis=0)
IPnumlist=IPnumlist.drop_duplicates()
IPnumlist.reset_index(drop=True, inplace=True)
IPnumi = IPnumlist.shape[0] 

time_start = time.time() 

#######Define Lists
Plist = []
Rlist = []
numlist = []
#######Pruning or not by changing i
finalkeep = Pruning(modelpred,i)

num = 1 ########Draw ROC and PRC curves according to the selection of num, That is, the number of considered abnormal records
#When num is 0, i.e. all records are considered normal  
#At this point R TPR FPR are 0, and P is not defined  
#So we define num to start from 1
while num < finalkeep.shape[0]+50:###+50 Ensure that all records can be obtained
    ab_finalsort= finalkeep[0:num]
    num += 50
    dfabip=pd.concat([ab_finalsort["source_ip"], ab_finalsort["destination_ip"]], axis=0)
    dfabip=dfabip.drop_duplicates()
    dfabip.reset_index(drop=True, inplace=True)
    if len(dfabip) == 0:
        continue
    dfabip.tolist()
    dfabip_set = set(dfabip)##Predicted set of abnormal IPs
    bingo=(dfabip_set)&(dfabcorrect_set)
    abIPi=len(dfabip_set)####Number of predicted abnormal IPs
    TPi=len(bingo)
    FNi=len(dfabcorrect)-TPi
    R,TPR,FPR,P = Metric(IPnumi,abIPi,TPi,FNi)
    
    #####Only Keep R==1
    if R==1:
        numlist.append(num-50)
        Plist.append(P)
        Rlist.append(R)

PRKeep = pd.DataFrame(Rlist, columns=['R'])
PRKeep = pd.concat([PRKeep, pd.DataFrame(Plist,columns=['P'])],axis=1)
PRKeep = pd.concat([PRKeep, pd.DataFrame(numlist,columns=['num'])],axis=1)


PRKeep.sort_values(by=['R','P'],inplace=True,ascending=True)
PRKeep.drop_duplicates(subset=['R'],keep='last', inplace=True)
PRKeep.reset_index(drop=True, inplace=True)

PRKeep

,R,P,num
0,1.0,0.619,380601


In [168]:
ab_final_output= finalkeep[0:PRKeep['num'][0]]
ab_final_output.to_csv("Results/WebFrontend_for_DFADR.csv",index=0)
PRKeep['num'][0]

380601